In [1]:
# Carregando as bibliotecas
import pandas as pd
import numpy as np

In [2]:
# Lendo os arquivos
veiculos_2017 = pd.read_csv('si-veic-2017.csv', sep=";", header=0, encoding='ISO-8859-1')
veiculos_2018 = pd.read_csv('si-veic-2018.csv', sep=";", header=0, encoding='ISO-8859-1')
veiculos_2019 = pd.read_csv('si-veic-2019.csv', sep=";", header=0, encoding='ISO-8859-1')

In [3]:
# Retirando o espaço em branco do nome das colunas
veiculos_2017 = veiculos_2017.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)
veiculos_2018 = veiculos_2018.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)
veiculos_2019 = veiculos_2019.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)

In [4]:
# Concatenando os arquivos de 2017 e 2018 em um único data frame
veiculos_df01 = pd.concat([veiculos_2017,veiculos_2018, veiculos_2019])
veiculos_df02=veiculos_df01.drop(columns=['data_hora_boletim', 'seq_veic', 'cod_categ',
       'descricao_categoria', 'cod_especie','cod_situacao', 'desc_situacao', 'tipo_socorro', 'desc_tipo_socorro'])
veiculos_df02.dropna(inplace=True)# Eliminando linhas em branco
print(veiculos_df02)

               Nº_boletim               descricao_especie
0      2017-012295590-001  MOTOCICLETA                   
1      2017-012295590-001  AUTOMOVEL                     
2      2017-012298672-001  MOTOCICLETA                   
3      2017-012298672-001  AUTOMOVEL                     
4      2017-012299194-001  MOTOCICLETA                   
...                   ...                             ...
24464  2019-006114270-001  ONIBUS                        
24465  2019-057581385-001  CAMINHAO                      
24466  2019-008607877-001  AUTOMOVEL                     
24467  2019-058608151-001  REBOQUE E SEMI-REBOQUE        
24468  2019-019022036-001  REBOQUE E SEMI-REBOQUE        

[67614 rows x 2 columns]


In [5]:
# Verificando os tipos de veiculos
veiculos_df02['descricao_especie']=veiculos_df02['descricao_especie'].str.strip() # Removendo os espaços em branco
tipos_veiculos = veiculos_df02['descricao_especie'].unique()
print(tipos_veiculos)

['MOTOCICLETA' 'AUTOMOVEL' 'MOTONETA' 'CAMIONETA' 'ONIBUS'
 'CAMINHAO-TRATOR' 'CAMINHONETE' 'MICROONIBUS' 'CAMINHAO' 'BICICLETA'
 'NAO INFORMADO' 'KOMBI' 'REBOQUE E SEMI-REBOQUE' 'CHARRETE' 'MISTO'
 'CICLOMOTOR' 'TRICICLO' 'CARROCA' 'TRATOR MISTO' 'CARRO DE MAO'
 'TRATOR DE RODAS' 'TRACAO' 'BONDE' 'ESPECIAL' 'PATINETE' 'TAXI']


In [6]:
# Verificando a ocorrencia por tipo de veiculo
ocorrencias_tipo = veiculos_df02['descricao_especie'].value_counts()
print(ocorrencias_tipo)

AUTOMOVEL                 31928
MOTOCICLETA               26591
ONIBUS                     3084
CAMINHONETE                1507
BICICLETA                  1171
CAMINHAO                   1049
CAMIONETA                   603
NAO INFORMADO               532
MOTONETA                    355
MICROONIBUS                 282
CAMINHAO-TRATOR             175
REBOQUE E SEMI-REBOQUE      165
KOMBI                        30
CARRO DE MAO                 29
CICLOMOTOR                   27
CARROCA                      25
PATINETE                     18
ESPECIAL                     11
BONDE                         8
TRATOR DE RODAS               7
TRICICLO                      5
CHARRETE                      3
TRACAO                        3
MISTO                         3
TRATOR MISTO                  2
TAXI                          1
Name: descricao_especie, dtype: int64


In [7]:
# Eliminando os tipos de veiculo não desejados
veiculos_df03 = veiculos_df02[~veiculos_df02['descricao_especie'].isin(['TRICICLO', 'CARROCA' ,'TRATOR MISTO' ,'CARRO DE MAO',
'TRATOR DE RODAS', 'TRACAO', 'BONDE', 'ESPECIAL', 'PATINETE', 'TAXI', 'NAO INFORMADO', 'KOMBI', 'REBOQUE E SEMI-REBOQUE',
'CHARRETE', 'MISTO', 'CICLOMOTOR'])]
ocorrencias_tipo01 = veiculos_df03['descricao_especie'].value_counts()
print(ocorrencias_tipo01)

AUTOMOVEL          31928
MOTOCICLETA        26591
ONIBUS              3084
CAMINHONETE         1507
BICICLETA           1171
CAMINHAO            1049
CAMIONETA            603
MOTONETA             355
MICROONIBUS          282
CAMINHAO-TRATOR      175
Name: descricao_especie, dtype: int64


In [8]:
# Reclassificando os veículos em BICICLETA, MOTOCICLETA, AUTOMOVEL, ONIBUS e CAMINHAO
veiculos_df04 = veiculos_df03.copy()
veiculos_df04.loc[:,'descricao_especie']  = veiculos_df04['descricao_especie'].replace({
    'MOTONETA' : 'MOTOCICLETA',
    'CAMIONETA' : 'AUTOMOVEL',
    'CAMINHONETE' : 'AUTOMOVEL',
    'CAMINHAO-TRATOR' : 'CAMINHAO',
    'MICROONIBUS' : 'ONIBUS'
})
print(veiculos_df04)

               Nº_boletim descricao_especie
0      2017-012295590-001       MOTOCICLETA
1      2017-012295590-001         AUTOMOVEL
2      2017-012298672-001       MOTOCICLETA
3      2017-012298672-001         AUTOMOVEL
4      2017-012299194-001       MOTOCICLETA
...                   ...               ...
24462  2019-061812147-001         AUTOMOVEL
24463  2019-032033144-001         AUTOMOVEL
24464  2019-006114270-001            ONIBUS
24465  2019-057581385-001          CAMINHAO
24466  2019-008607877-001         AUTOMOVEL

[66745 rows x 2 columns]


In [9]:
# Criando colunas por tipo de veículos
veiculos_df05 = veiculos_df04.copy()
veiculos_df05['BICICLETA']=np.where(veiculos_df05['descricao_especie']=='BICICLETA',1,0)
veiculos_df05['MOTOCICLETA']=np.where(veiculos_df05['descricao_especie']=='MOTOCICLETA',1,0)
veiculos_df05['AUTOMOVEL']=np.where(veiculos_df05['descricao_especie']=='AUTOMOVEL',1,0)
veiculos_df05['CAMINHAO']=np.where(veiculos_df05['descricao_especie']=='CAMINHAO',1,0)
veiculos_df05['ONIBUS']=np.where(veiculos_df05['descricao_especie']=='ONIBUS',1,0)
print(veiculos_df05)

               Nº_boletim descricao_especie  BICICLETA  MOTOCICLETA  \
0      2017-012295590-001       MOTOCICLETA          0            1   
1      2017-012295590-001         AUTOMOVEL          0            0   
2      2017-012298672-001       MOTOCICLETA          0            1   
3      2017-012298672-001         AUTOMOVEL          0            0   
4      2017-012299194-001       MOTOCICLETA          0            1   
...                   ...               ...        ...          ...   
24462  2019-061812147-001         AUTOMOVEL          0            0   
24463  2019-032033144-001         AUTOMOVEL          0            0   
24464  2019-006114270-001            ONIBUS          0            0   
24465  2019-057581385-001          CAMINHAO          0            0   
24466  2019-008607877-001         AUTOMOVEL          0            0   

       AUTOMOVEL  CAMINHAO  ONIBUS  
0              0         0       0  
1              1         0       0  
2              0         0       0  

In [10]:
# Excluindo a coluna descricao _especie
veiculos_df06 = veiculos_df05.drop(columns=['descricao_especie'])
veiculos_df06 = veiculos_df06.rename(columns={'Nº_boletim': 'NUMERO_BOLETIM'})
print(veiculos_df06)

           NUMERO_BOLETIM  BICICLETA  MOTOCICLETA  AUTOMOVEL  CAMINHAO  ONIBUS
0      2017-012295590-001          0            1          0         0       0
1      2017-012295590-001          0            0          1         0       0
2      2017-012298672-001          0            1          0         0       0
3      2017-012298672-001          0            0          1         0       0
4      2017-012299194-001          0            1          0         0       0
...                   ...        ...          ...        ...       ...     ...
24462  2019-061812147-001          0            0          1         0       0
24463  2019-032033144-001          0            0          1         0       0
24464  2019-006114270-001          0            0          0         0       1
24465  2019-057581385-001          0            0          0         1       0
24466  2019-008607877-001          0            0          1         0       0

[66745 rows x 6 columns]


In [11]:
# Agrupando pelo Numero do boletim
veiculos_df07=veiculos_df06.copy()
veiculos_df07=veiculos_df07.groupby('NUMERO_BOLETIM', as_index=False).sum()
print(veiculos_df07)

           NUMERO_BOLETIM  BICICLETA  MOTOCICLETA  AUTOMOVEL  CAMINHAO  ONIBUS
0      2017-000002733-001          0            2          0         0       0
1      2017-000015078-001          0            0          1         0       0
2      2017-000015654-001          0            1          1         0       0
3      2017-000016477-001          0            1          0         0       0
4      2017-000020133-001          0            0          1         0       0
...                   ...        ...          ...        ...       ...     ...
37124  2020-012578761-001          0            1          0         0       0
37125  2020-012581709-001          0            1          1         0       0
37126  2020-013714143-001          0            1          0         0       0
37127  2020-016593364-001          0            0          0         1       0
37128  2020-016807193-001          0            1          1         0       0

[37129 rows x 6 columns]


In [12]:
veiculos_df07.to_excel('Base_Veículos.xlsx', index=False)